<a href="https://colab.research.google.com/github/abdulR1806/BSS24-25/blob/main/notebook_yield_branch_exp.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np

# --- 1. Load your Data ---
# Assume 'df' is your main DataFrame containing the data from the first image.
# Replace this with your actual data loading (e.g., pd.read_csv, pd.read_excel)
# Example using dummy data based on the structure:
data = {
    'Periode': pd.to_datetime(['2023-01-31', '2023-01-31', '2023-01-31', '2023-01-31', '2023-02-28', '2023-02-28']),
    'Branch': ['Branch A', 'Branch B', 'Branch A', 'Branch C', 'Branch B', 'Branch A'],
    'Branch_Flag': ['Flag1', 'Flag2', 'Flag1', 'Flag3', 'Flag2', 'Flag1'],
    'ID': ['ID001', 'ID002', 'ID003', 'ID004', 'ID005', 'ID006'],
    'DPD90_Amt': [0, 5000, 1000, 0, 8000, 0],
    'NEA': [100000, 150000, 80000, 200000, 160000, 120000],
    'Int_Branch': [1000, 1200, 900, 1500, 1300, 1100],
    'AEA': [95000, 140000, 78000, 190000, 155000, 115000],
    'RSC_Amt': [0, 10000, 0, 5000, 15000, 0],
    'SM_Amt': [20000, 30000, 15000, 40000, 35000, 25000],
    'NonSM_Amt': [80000, 110000, 65000, 155000, 110000, 90000],
    'War_Amt': [500, 600, 400, 700, 650, 550],
    'Pre_Covid_Amt': [100000, 140000, 80000, 195000, 145000, 120000],
    'Post_Covid_Amt': [0, 10000, 0, 5000, 15000, 0],
    'OS_LTV': [70000, 90000, 60000, 120000, 100000, 80000],
    'MARKET VALUE': [100000, 110000, 90000, 150000, 130000, 100000], # Note the space in the name
    'DPD30_Amt': [5000, 6000, 2000, 3000, 9000, 1000],
    'War_Amt_Ori': [550, 650, 450, 750, 700, 600]
    # Add other columns as needed from your first image
}
df = pd.DataFrame(data)

# It's often easier if column names don't have spaces. Rename if possible:
if 'MARKET VALUE' in df.columns:
    df = df.rename(columns={'MARKET VALUE': 'MARKET_VALUE'})
    market_value_col = 'MARKET_VALUE'
else:
    # Handle case where renaming didn't happen or wasn't needed
    market_value_col = 'MARKET VALUE' if 'MARKET VALUE' in df.columns else 'MARKET_VALUE'


# --- 2. Define Columns to Aggregate ---
# List all columns mentioned in the formulas that need to be summed per branch
aggregation_cols = [
    'DPD90_Amt', 'NEA', 'Int_Branch', 'AEA', 'RSC_Amt', 'SM_Amt',
    'NonSM_Amt', 'War_Amt', 'Pre_Covid_Amt', 'Post_Covid_Amt',
    'OS_LTV', market_value_col, 'DPD30_Amt', 'War_Amt_Ori'
]

# --- 3. Group by Branch and Aggregate ---
# Calculate the sum of the necessary columns for each branch
# Using .reset_index() makes 'Branch' a column again, which can be convenient
branch_aggregates = df.groupby('Branch')[aggregation_cols].sum().reset_index()

# --- 4. Define Safe Division Function ---
# This function replicates the IFERROR(calculation, 0) logic
def safe_divide(numerator, denominator):
    """Performs division, returning 0 if denominator is 0 or NaN, or if numerator is NaN."""
    # Ensure inputs work correctly even if they are single numbers from aggregation
    num = pd.Series(numerator)
    den = pd.Series(denominator)
    result = pd.Series(np.zeros(len(num)), index=num.index) # Initialize with zeros
    # Create a mask for valid division (denominator > 0 and not NaN, numerator not NaN)
    valid_mask = (den != 0) & (den.notna()) & (num.notna())
    # Perform division only for valid entries
    result[valid_mask] = num[valid_mask] / den[valid_mask]
    return result

# --- 5. Calculate Metrics per Branch ---
calculated_metrics = pd.DataFrame()
calculated_metrics['Branch'] = branch_aggregates['Branch'] # Start with the Branch column

# Apply formulas using the aggregated data and safe_divide
calculated_metrics['NPL'] = safe_divide(branch_aggregates['DPD90_Amt'], branch_aggregates['NEA'])
calculated_metrics['Yield'] = safe_divide(branch_aggregates['Int_Branch'], branch_aggregates['AEA']) * 12
calculated_metrics['Restructured'] = safe_divide(branch_aggregates['RSC_Amt'], branch_aggregates['NEA'])
calculated_metrics['S_Merdeka'] = safe_divide(branch_aggregates['SM_Amt'], branch_aggregates['NEA'])
calculated_metrics['NonSM'] = safe_divide(branch_aggregates['NonSM_Amt'], branch_aggregates['NEA'])
calculated_metrics['War'] = safe_divide(branch_aggregates['War_Amt'], branch_aggregates['AEA']) * 12
calculated_metrics['Pre_Covid'] = safe_divide(branch_aggregates['Pre_Covid_Amt'], branch_aggregates['NEA'])
calculated_metrics['Post_Covid'] = safe_divide(branch_aggregates['Post_Covid_Amt'], branch_aggregates['NEA'])
calculated_metrics['LTV%'] = safe_divide(branch_aggregates['OS_LTV'], branch_aggregates[market_value_col])
calculated_metrics['30+'] = safe_divide(branch_aggregates['DPD30_Amt'], branch_aggregates['NEA'])
calculated_metrics['WAR ORI'] = safe_divide(branch_aggregates['War_Amt_Ori'], branch_aggregates['AEA']) * 12

# --- 6. Display Results ---
# Set Branch as the index for clearer output
calculated_metrics = calculated_metrics.set_index('Branch')

# Optional: Reorder columns based on the 'Solve Order' if desired
solve_order_cols = ['NPL', 'Yield', 'Restructured', 'S_Merdeka', 'NonSM', 'War',
                    'Pre_Covid', 'Post_Covid', 'LTV%', '30+', 'WAR ORI']
# Ensure all calculated columns exist before reordering
existing_cols = [col for col in solve_order_cols if col in calculated_metrics.columns]
calculated_metrics = calculated_metrics[existing_cols]


print("Calculated Metrics Grouped by Branch:")
print(calculated_metrics)

Calculated Metrics Grouped by Branch:
               NPL     Yield  Restructured  S_Merdeka     NonSM       War  \
Branch                                                                      
Branch A  0.003333  0.125000      0.000000   0.200000  0.783333  0.060417   
Branch B  0.041935  0.101695      0.080645   0.209677  0.709677  0.050847   
Branch C  0.000000  0.094737      0.025000   0.200000  0.775000  0.044211   

          Pre_Covid  Post_Covid      LTV%       30+   WAR ORI  
Branch                                                         
Branch A   1.000000    0.000000  0.724138  0.026667  0.066667  
Branch B   0.919355    0.080645  0.791667  0.048387  0.054915  
Branch C   0.975000    0.025000  0.800000  0.015000  0.047368  


In [ ]:
import pandas as pd
import numpy as np

# --- 0. Shared Setup & Data Loading ---
# Assume 'df' is your main DataFrame with all necessary columns.

# Example using dummy data (ensure all relevant columns are present):
data = {
    'Periode': pd.to_datetime(['2023-01-31', '2023-01-31', '2023-01-31', '2023-01-31', '2023-02-28', '2023-02-28', '2023-03-31', '2023-03-31', '2023-01-31',
                               '2023-02-28', '2023-02-28', '2023-03-31']), # Added more data for next month calc
    'Branch': ['Branch A', 'Branch B', 'Branch A', 'Branch C', 'Branch B', 'Branch A', 'Branch A', 'Branch B', 'Branch A',
               'Branch A', 'Branch C', 'Branch B'], # Added more data
    'Type':   ['Type X', 'Type Y', 'Type Y', 'Type X', 'Type Y', 'Type X', 'Type X', 'Type Y', 'Type X',
               'Type Y', 'Type X', 'Type Y'], # Added more data
    'CIF_Open_Date': pd.to_datetime(['2022-05-10', '2021-11-20', '2022-08-01', '2022-01-15', '2021-12-01', '2023-01-05', '2022-06-30', '2023-02-10', '2021-09-15',
                                     '2022-03-10', '2023-01-20','2022-09-01']), # Added more data
    'Plafond_Tier': ['Tier 1', 'Tier 2', 'Tier 1', 'Tier 3', 'Tier 2', 'Tier 1', 'Tier 2', 'Tier 2', 'Tier 3',
                     'Tier 1', 'Tier 3', 'Tier 2'], # Added more data
    'Interest_Tier': ['Low', 'Medium', 'Low', 'High', 'Medium', 'Low', 'Medium', 'Low', 'High',
                      'Medium', 'High', 'Low'], # Interest rate tiers
    'ID': [f'ID{i:03d}' for i in range(1, 13)],
    'DPD90_Amt': [0, 5000, 1000, 0, 8000, 0, 200, 6000, 50, 100, 0, 7500],
    'NEA': [100000, 150000, 80000, 200000, 160000, 120000, 110000, 140000, 90000, 85000, 210000, 145000],
    'Int_Branch': [1000, 1200, 900, 1500, 1300, 1100, 1050, 1150, 950, 800, 1600, 1180],
    'AEA': [95000, 140000, 78000, 190000, 155000, 115000, 108000, 135000, 88000, 82000, 205000, 142000],
    'RSC_Amt': [0, 10000, 0, 5000, 15000, 0, 1000, 8000, 200, 0, 6000, 9000],
    'SM_Amt': [20000, 30000, 15000, 40000, 35000, 25000, 22000, 28000, 18000, 16000, 45000, 29000],
    'NonSM_Amt': [80000, 110000, 65000, 155000, 110000, 90000, 87000, 104000, 71800, 69000, 159000, 107000],
    'War_Amt': [500, 600, 400, 700, 650, 550, 520, 580, 480, 450, 720, 590],
    'Pre_Covid_Amt': [100000, 140000, 80000, 195000, 145000, 120000, 109000, 132000, 90000, 85000, 204000, 136000],
    'Post_Covid_Amt': [0, 10000, 0, 5000, 15000, 0, 1000, 8000, 200, 0, 6000, 9000],
    'OS_LTV': [70000, 90000, 60000, 120000, 100000, 80000, 75000, 85000, 65000, 60000, 130000, 90000],
    'MARKET VALUE': [100000, 110000, 90000, 150000, 130000, 100000, 105000, 100000, 95000, 90000, 160000, 110000],
    'DPD30_Amt': [5000, 6000, 2000, 3000, 9000, 1000, 1500, 7000, 800, 900, 3500, 8000],
    'War_Amt_Ori': [550, 650, 450, 750, 700, 600, 570, 630, 500, 480, 780, 610]
}
df = pd.DataFrame(data)

# --- Basic Prep (as before) ---
if 'Type' not in df.columns: raise ValueError("Missing 'Type' column.")
if 'Plafond_Tier' not in df.columns: raise ValueError("Missing 'Plafond_Tier' column.")
if 'Interest_Tier' not in df.columns: raise ValueError("Missing 'Interest_Tier' column.")
cif_date_col = 'CIF_Open_Date'
if cif_date_col not in df.columns: raise ValueError(f"Missing '{cif_date_col}' column.")

df['Periode'] = pd.to_datetime(df['Periode']) # Ensure Periode is datetime
df[cif_date_col] = pd.to_datetime(df[cif_date_col], errors='coerce')
df['CIF_Booking_Year'] = df[cif_date_col].dt.year.astype('Int64')
df['Plafond_Tier'] = df['Plafond_Tier'].astype(str)
df['Interest_Tier'] = df['Interest_Tier'].astype(str)
df['Type'] = df['Type'].astype(str)

if 'MARKET VALUE' in df.columns:
    df = df.rename(columns={'MARKET VALUE': 'MARKET_VALUE'})
    market_value_col = 'MARKET_VALUE'
else:
    market_value_col = 'MARKET VALUE' if 'MARKET VALUE' in df.columns else 'MARKET_VALUE'

# --- Define Safe Divide & Aggregation Dict ---
def safe_divide(numerator, denominator):
    num = pd.Series(numerator)
    den = pd.Series(denominator)
    result = pd.Series(np.zeros(len(num)), index=num.index)
    valid_mask = (den != 0) & (den.notna()) & (num.notna())
    result[valid_mask] = num[valid_mask] / den[valid_mask]
    if len(result) == 1: return result.iloc[0]
    return result

aggregation_cols_sum = ['DPD90_Amt', 'NEA', 'Int_Branch', 'AEA', 'RSC_Amt', 'SM_Amt', 'NonSM_Amt', 'War_Amt', 'Pre_Covid_Amt', 'Post_Covid_Amt', 'OS_LTV', market_value_col, 'DPD30_Amt', 'War_Amt_Ori']
agg_dict = {col: 'sum' for col in aggregation_cols_sum}
agg_dict['ID'] = 'count'

# --- Calculate Period Totals ---
period_totals = df.groupby('Periode')['NEA'].sum().reset_index()
period_totals = period_totals.rename(columns={'NEA': 'Total_NEA_Period'})

# --- Function to Calculate Metrics ---
def calculate_metrics_grouped(df_grouped, grouping_cols):
    # Ensure Total_NEA_Period is present after merge
    if 'Total_NEA_Period' not in df_grouped.columns:
         # This might happen if called before merging period_totals
         raise ValueError("Total_NEA_Period column missing in grouped data before metric calculation.")

    calculated_metrics = pd.DataFrame()
    for col in grouping_cols:
        calculated_metrics[col] = df_grouped[col]

    # Calculate metrics... (same calculation logic as before)
    calculated_metrics['NPL'] = safe_divide(df_grouped['DPD90_Amt'], df_grouped['NEA'])
    calculated_metrics['Yield'] = safe_divide(df_grouped['Int_Branch'], df_grouped['AEA']) * 12
    calculated_metrics['Restructured'] = safe_divide(df_grouped['RSC_Amt'], df_grouped['NEA'])
    calculated_metrics['S_Merdeka'] = safe_divide(df_grouped['SM_Amt'], df_grouped['NEA'])
    calculated_metrics['NonSM'] = safe_divide(df_grouped['NonSM_Amt'], df_grouped['NEA'])
    calculated_metrics['War'] = safe_divide(df_grouped['War_Amt'], df_grouped['AEA']) * 12
    calculated_metrics['Pre_Covid_Ratio'] = safe_divide(df_grouped['Pre_Covid_Amt'], df_grouped['NEA'])
    calculated_metrics['Post_Covid_Ratio'] = safe_divide(df_grouped['Post_Covid_Amt'], df_grouped['NEA'])
    calculated_metrics['LTV%'] = safe_divide(df_grouped['OS_LTV'], df_grouped[market_value_col])
    calculated_metrics['30+'] = safe_divide(df_grouped['DPD30_Amt'], df_grouped['NEA'])
    calculated_metrics['WAR ORI'] = safe_divide(df_grouped['War_Amt_Ori'], df_grouped['AEA']) * 12
    calculated_metrics['Sum_NEA'] = df_grouped['NEA']
    calculated_metrics['Sum_RSC_Amt'] = df_grouped['RSC_Amt']
    calculated_metrics['Sum_Pre_Covid_Amt'] = df_grouped['Pre_Covid_Amt']
    calculated_metrics['Percentage_NEA_of_Total'] = safe_divide(df_grouped['NEA'], df_grouped['Total_NEA_Period']) * 100
    calculated_metrics['Count_of_ID'] = df_grouped['Count_of_ID']
    return calculated_metrics

# --- Generate the 3 Pivoted DataFrames (A, B, C as before) ---

# A: Pivoted by Type
grouping_keys_type = ['Branch', 'Periode', 'Type']
branch_period_type_aggregates = df.groupby(grouping_keys_type).agg(agg_dict).reset_index()
branch_period_type_aggregates = branch_period_type_aggregates.rename(columns={'ID': 'Count_of_ID'})
branch_period_type_aggregates = pd.merge(branch_period_type_aggregates, period_totals, on='Periode', how='left')
calculated_metrics_by_type = calculate_metrics_grouped(branch_period_type_aggregates, grouping_keys_type)
calculated_metrics_by_type = calculated_metrics_by_type.set_index(grouping_keys_type)
df_pivoted = calculated_metrics_by_type.unstack(level='Type')
df_pivoted.columns = ['_'.join(col).strip().replace(' ', '_') for col in df_pivoted.columns.values]

# B: Pivoted by Year
grouping_keys_year = ['Branch', 'Periode', 'CIF_Booking_Year']
branch_period_year_aggregates = df.dropna(subset=['CIF_Booking_Year']).groupby(grouping_keys_year).agg(agg_dict).reset_index()
branch_period_year_aggregates = branch_period_year_aggregates.rename(columns={'ID': 'Count_of_ID'})
branch_period_year_aggregates = pd.merge(branch_period_year_aggregates, period_totals, on='Periode', how='left')
calculated_metrics_by_year = calculate_metrics_grouped(branch_period_year_aggregates, grouping_keys_year)
calculated_metrics_by_year['CIF_Booking_Year'] = calculated_metrics_by_year['CIF_Booking_Year'].astype(int)
calculated_metrics_by_year = calculated_metrics_by_year.set_index(grouping_keys_year)
pivoted_by_year_metrics = calculated_metrics_by_year.unstack(level='CIF_Booking_Year')
pivoted_by_year_metrics.columns = [f'{metric}_{year}' for metric, year in pivoted_by_year_metrics.columns]

# C: Pivoted by Plafond Tier
plafond_col = 'Plafond_Tier'
grouping_keys_plafond = ['Branch', 'Periode', plafond_col]
branch_period_plafond_aggregates = df.dropna(subset=[plafond_col]).groupby(grouping_keys_plafond).agg(agg_dict).reset_index()
branch_period_plafond_aggregates = branch_period_plafond_aggregates.rename(columns={'ID': 'Count_of_ID'})
branch_period_plafond_aggregates = pd.merge(branch_period_plafond_aggregates, period_totals, on='Periode', how='left')
calculated_metrics_by_plafond = calculate_metrics_grouped(branch_period_plafond_aggregates, grouping_keys_plafond)
calculated_metrics_by_plafond = calculated_metrics_by_plafond.set_index(grouping_keys_plafond)
pivoted_by_plafond_metrics = calculated_metrics_by_plafond.unstack(level=plafond_col)
pivoted_by_plafond_metrics.columns = [f'{metric}_{tier.replace(" ", "_")}' for metric, tier in pivoted_by_plafond_metrics.columns]

# D: Pivoted by Interest Tier
interest_col = 'Interest_Tier'
grouping_keys_interest = ['Branch', 'Periode', interest_col]
branch_period_interest_aggregates = df.dropna(subset=[interest_col]).groupby(grouping_keys_interest).agg(agg_dict).reset_index()
branch_period_interest_aggregates = branch_period_interest_aggregates.rename(columns={'ID': 'Count_of_ID'})
branch_period_interest_aggregates = pd.merge(branch_period_interest_aggregates, period_totals, on='Periode', how='left')
calculated_metrics_by_interest = calculate_metrics_grouped(branch_period_interest_aggregates, grouping_keys_interest)
calculated_metrics_by_interest = calculated_metrics_by_interest.set_index(grouping_keys_interest)
pivoted_by_interest_metrics = calculated_metrics_by_interest.unstack(level=interest_col)
pivoted_by_interest_metrics.columns = [f'{metric}_{tier.replace(" ", "_")}' for metric, tier in pivoted_by_interest_metrics.columns]

# --- E: Combine the four pivoted DataFrames ---
combined_df_step1 = pd.merge(df_pivoted, pivoted_by_year_metrics, left_index=True, right_index=True, how='outer')
combined_df_step2 = pd.merge(combined_df_step1, pivoted_by_plafond_metrics, left_index=True, right_index=True, how='outer')
final_combined_df = pd.merge(combined_df_step2, pivoted_by_interest_metrics, left_index=True, right_index=True, how='outer')
print(f"Shape of combined pivoted data: {final_combined_df.shape}")


# --- F: Generate Original Aggregated Data (Grouped only by Branch, Period) ---
print("\n--- Generating Original Aggregated Data (Branch, Periode) ---")
grouping_keys_overall = ['Branch', 'Periode']
overall_aggregates = df.groupby(grouping_keys_overall).agg(agg_dict).reset_index()
overall_aggregates = overall_aggregates.rename(columns={'ID': 'Count_of_ID'})
# Merge period totals needed for percentage calculation
overall_aggregates = pd.merge(overall_aggregates, period_totals, on='Periode', how='left')

# Calculate metrics for the overall group
original_aggregated_metrics = calculate_metrics_grouped(overall_aggregates, grouping_keys_overall)

# Rename columns to avoid clashes with pivoted columns
original_aggregated_metrics.columns = [f"{col}_Overall" if col not in grouping_keys_overall else col for col in original_aggregated_metrics.columns]

# Set index for merging
original_aggregated_metrics = original_aggregated_metrics.set_index(grouping_keys_overall)
print(f"Shape of original aggregated data: {original_aggregated_metrics.shape}")


# --- G: Merge Original Aggregated Data with Combined Pivoted Data ---
print("\n--- Merging Original Aggregated Data with Pivoted Data ---")
final_dataset = pd.merge(
    final_combined_df,
    original_aggregated_metrics,
    left_index=True,
    right_index=True,
    how='left' # Keep all rows from the pivoted table, add overall metrics
)
print(f"Shape after final merge: {final_dataset.shape}")


# --- H: Create Target Variables ---
print("\n--- Creating Target Variables ---")
# Sort by Branch then Period to ensure correct shifting
final_dataset = final_dataset.sort_index(level=['Branch', 'Periode'])

# Shift the 'Yield_Overall' column within each Branch group
final_dataset['Yield_Overall_NextMonth'] = final_dataset.groupby('Branch')['Yield_Overall'].shift(-1)
final_dataset['Yield_Decrease_Next_Month'] = (final_dataset['Yield_Overall_NextMonth'] < final_dataset['Yield_Overall']).astype(int)

# Shift the 'NPL_Overall' column (or '30+_Overall' if preferred)
final_dataset['NPL_Overall_NextMonth'] = final_dataset.groupby('Branch')['NPL_Overall'].shift(-1)
final_dataset['DPD_Increase_Next_Month'] = (final_dataset['NPL_Overall_NextMonth'] > final_dataset['NPL_Overall']).astype(int)

# Optionally drop the intermediate shifted columns
# final_dataset = final_dataset.drop(columns=['Yield_Overall_NextMonth', 'NPL_Overall_NextMonth'])

# --- I: Display Final Result ---
print("\n--- Final Dataset with Target Variables ---")
# Select key columns including new targets to display snippet
display_cols = list(final_dataset.filter(like='_Overall').columns) + ['Yield_Decrease_Next_Month', 'DPD_Increase_Next_Month']
print(final_dataset[display_cols].to_string())

# For full view:
# print(final_dataset.info())
# print(final_dataset.to_string())

Shape of combined pivoted data: (8, 128)

--- Generating Original Aggregated Data (Branch, Periode) ---
Shape of original aggregated data: (8, 16)

--- Merging Original Aggregated Data with Pivoted Data ---
Shape after final merge: (8, 144)

--- Creating Target Variables ---

--- Final Dataset with Target Variables ---
                     NPL_Overall  Yield_Overall  Restructured_Overall  S_Merdeka_Overall  NonSM_Overall  War_Overall  Pre_Covid_Ratio_Overall  Post_Covid_Ratio_Overall  LTV%_Overall  30+_Overall  WAR ORI_Overall  Sum_NEA_Overall  Sum_RSC_Amt_Overall  Sum_Pre_Covid_Amt_Overall  Percentage_NEA_of_Total_Overall  Count_of_ID_Overall  Yield_Overall_NextMonth  NPL_Overall_NextMonth  Yield_Decrease_Next_Month  DPD_Increase_Next_Month
Branch   Periode                                                                                                                                                                                                                                        

In [ ]:
len(final_dataset.columns.to_list())

148